# Web Scraper

Originalmente mi plan era obtener la información de cada página de una expansión de cartas donde existen más de 65 expansiones en un catálogo. La primera caja de código sólo tiene la araña que usamos en una lección anterior que nos permitía iterar sobre varías páginas para obtener el texto. Pensaba usarla para obtener la información de cada tabla de cartas de todas las expansiones.  

In [4]:
import requests
from bs4 import BeautifulSoup
import time
import re


class IronhackSpider:
    
    def __init__(self, url_pattern, pages_to_scrape=10, sleep_interval=-1, content_parser=None):
        self.url_pattern = url_pattern
        self.pages_to_scrape = pages_to_scrape
        self.sleep_interval = sleep_interval
        self.content_parser = content_parser
    

    def scrape_url(self, url):
        try:
            response = requests.get(url)
            result = self.content_parser(response.content)
            self.output_results(result)
        except exception as excep:
            print(type(excep), excep)
            print('ERROR!')
    
   
    def output_results(self, r):
        print(r)
    

    def kickstart(self, sleep_interval = 1):
        if self.sleep_interval > 0:  
            for i in range(1, self.pages_to_scrape+1):
                self.scrape_url(self.url_pattern % i)
                time.sleep(self.sleep_interval)


URL_PATTERN = 'http://quotes.toscrape.com/page/%s/' # regex pattern for the urls to scrape
PAGES_TO_SCRAPE = 1 # how many webpages to scrapge

def quotes_parser(content):
    return content

# Instantiate the IronhackSpider class
my_spider = IronhackSpider(URL_PATTERN, PAGES_TO_SCRAPE, content_parser=quotes_parser)

# Start scraping jobs
# my_spider.kickstart()

In [ ]:
# Código donde intente obtener el link de cada expansión no funcionó. 

'''URL_PATTERN = 'http://www.starcitygames.com/catalog/magic_the_gathering/'
PAGES_TO_SCRAPE = 1

def quotes_parser(content):
    content = BeautifulSoup(content, 'html')
    nueva_sopa = content.find_all('ul', {'class':'cardset_lists'})
    text = [i.get_text()for i in nueva_sopa]
    print(text)
    return text
quotes_parser = quotes_parser
my_spider = IronhackSpider(URL_PATTERN, PAGES_TO_SCRAPE, content_parser=quotes_parser) 
my_spider.kickstart()'''

In [6]:
# Código donde obtuve los links de cada expansión, sí funcionó. 
URL_PATTERN = 'http://www.starcitygames.com/catalog/magic_the_gathering/'
PAGES_TO_SCRAPE = 1

def quotes_parser(content):
    catalog_web = BeautifulSoup(content, 'html')
    links_to_categories = catalog_web.select('ul.cardset_lists a[href^= "http://www.starcitygames.com/catalog/category/"]')
    links = [link['href'] for link in links_to_categories]
    return links

quotes_parser = quotes_parser
my_spider = IronhackSpider(URL_PATTERN, PAGES_TO_SCRAPE, content_parser=quotes_parser) 
my_spider.kickstart()

NameError: name 'content' is not defined

In [ ]:
# Código donde intenté obtener la información de la tabla con los datos de cada carta y su precio. Si funcionó pero 
# la información que obtenía era difícil de manejar y no tenía que hacerle muchas limpieza. 

'''
catalog = requests.get('http://www.starcitygames.com/catalog/category/Oath%20of%20the%20Gatewatch').content
catalog_web = BeautifulSoup(catalog, 'html')
#print(catalog_web.prettify())

links_to_categories = catalog_web.select('tr.deckdbbody_row, deckdbbody2_row:nth-child(1)')
#links_to_categories = catalog_web.find_all('td', {'class': re.compile('search_results_9$')})
print(links_to_categories)
#links_to_categories = [link.text for link in links_to_categories]

#print(tmp3)
links = [re.sub(r'[\n\n\n\n\n\n\n\n]', '    ', link.text).strip(' , \xa0').split('    ') for link in links_to_categories]
#print(links)'''

In [ ]:
# Otro intento de obtener la información de la tabla de datos y precios de las cartas. Esta vez intente obtenerlas 
# accediendo a las columnas. 
catalog = requests.get('http://www.starcitygames.com/catalog/category/Oath%20of%20the%20Gatewatch').content
catalog_web = BeautifulSoup(catalog, 'html')
#print(catalog_web.prettify())

# Obtiene el nombre de las cartas. Resulto problemático porque hay cartas que tienen tres filas pero sólo tienen una vez
# el nombre lo cual desconfigura la información. 
card_name = catalog_web.find_all('td', {'class': 'deckdbbody search_results_1'})
card_name = [link.text for link in card_name]
#print(card_name)

# obtiene el nombre de la expansión a la que pertenece la carta. 
expansion = catalog_web.find_all('td', {'class': 'deckdbbody search_results_2'})
expansion = [link.text for link in expansion]
#print(expansion)

# obtiene el tipo de la carta. 
card_type = catalog_web.find_all('td', {'class': 'deckdbbody search_results_4'})
card_type = [link.text for link in card_type]
#print(card_type)
  
# obtiene la rareza de la carta. 
rarity = catalog_web.find_all('td', {'class': 'deckdbbody search_results_6'})
rarity = [link.text for link in rarity]
#print(rarity)

# Justo el lugar más problemático junto con el nombre. Para una carta había más de un precio y como no existía una relación 
# con el nombre y había precios que tenían descuento, se registraba el precio original y el descuento, juntar las columnas
# iba a ser una verdadera pesadilla, más si pensaba iterar esto en varias páginas. 
price = catalog_web.select('tr > td:')
#price = [link.text for link in price]
for element in price:
    print(element)
    print('\n')


In [96]:
# Aquí Danilo hizo su magia y convirtió directamente la página a una data frame de pandas. Esto sólo fue posible porque
# la página estaba construida tabla sobre tabla. Esta función es útil para entrar a las tablas. 
import pandas as pd

catalog = requests.get('http://www.starcitygames.com/catalog/category/Oath%20of%20the%20Gatewatch').content
# Función de pandas que lee directamente un html 
table = pd.read_html(catalog)
# La tabla contiene varias tablas antes de elegir visualizamos que contenía cada una. La tabla indice 1 resultó la mejor
# para lo que quería hacer
df_table = table[1]


In [97]:
# Esta parte del código hace la limpieza del data frame ya seleccionado. 

table = pd.read_html(catalog)
df_table = table[1]

# elimina y renombra basado en una fila. 
df_table = df_table.drop(0) # elimina la primera fila 
name = df_table.iloc[0,] # selecciona la fila donde están los nombres de la columna y los guarda en una variable
df_table = df_table.rename(name, axis = 1) # utiliza la variable de los nombres y renombra las columnas. 
df_table = df_table.drop(1) # elimina la fila que contenía los nombres, ya no se necesita. 
df_table = df_table.drop(['Mana', 'Stock'], axis = 1) # elimina dos columnas que no me interesan. 

#df_table.columns[7] revisa la columna por indice porque no se podía acceder mediante la selección de nombre
df_table = df_table.drop(df_table.columns[7],  axis = 1) # elimina la última columna del data frame que no contiene nada y esta conformada por NaNs
df_table = df_table.dropna(axis = 0, subset = ['Name', 'Category']) # Elimina columnas si tienen valores Nan en las columnas indicadas.
df_table = df_table.reset_index(drop = True) # vuelve a colocar la indexación desde 0 y elimina la anterior si drop = True

for doble_price in df_table.Price: #recorre la columna de precio y encuentra el valor del precio más a la derecha, que en caso de que tenga descuento 
    fix_price = re.findall('..\...$', doble_price) # es el precio con descuento sino es el valor original.  
    df_table.Price = df_table.Price.replace({doble_price: fix_price}) # reemplaza el precio en el lugar donde lo recorrió y actualiza el data frame.

df_table.to_csv('tabla_precios_mtg.csv', index = False) # Lo convierte en un csv para su manejo. 

In [79]:
# Busca el link del botón next que lleva a la siguiente página del set. 

card_page = requests.get('http://www.starcitygames.com/catalog/category/Oath%20of%20the%20Gatewatch').content
card_page = BeautifulSoup(card_page, 'html')
next_link = card_page.select('section.box_sizing_fix#content div[style] a:contains(Next)')
next_link = next_link[0]['href']
next_link

'http://www.starcitygames.com/catalog/category/5315?&start=50'

### El siguiente código es mi versión de la araña para poder iterar sobre todas las expansiones pero no funcionó y por lo menos ahorita no se me ocurre cómo hacerle. Me gustaría que me ayudaran con esta parte. 


In [92]:
URL_PATTERN = 'http://www.starcitygames.com/catalog/category/'
PAGES_TO_SCRAPE = links


class IronhackSpiderMagic:
    
    def __init__(self, pages_to_scrape, next_page, sleep_interval=-1, content_parser=None):
        self.pages_to_scrape = pages_to_scrape
        self.sleep_interval = sleep_interval
        self.content_parser = content_parser
        self.next_page = next_page


    def scrape_url(self, url):
            next_link = 'link' 
            while len(next_link) > 0:  
                try:
                    response = requests.get(url).content
                    result = self.content_parser(response)
                    pre_data = self.output_results(result)
                    link = self.next_page(response)
                except exception as excep:
                    print(type(excep), excep)
                    print('ERROR!')
            else:            
                pre_data = pd.concat(pre_data, ignore_index = True)
                print(pre_data)
    
    def next_page(self, content):
        card_page = BeautifulSoup(content, 'html')
        next_link = card_page.select('section.box_sizing_fix#content div[style] a:contains(Next)')
        next_link = next_link[0]['href']
        return next_link
                
    def output_results(self, combined):
        pre_data = []
        pre_data.append(combined)
        return pre_data
                
        

    def kickstart(self, sleep_interval = 0):
            if self.sleep_interval > 0:  
                for url in range(self.pages_to_scrape):
                    self.scrape_url(url)
                    time.sleep(self.sleep_interval)



def card_parser(content):
    return content
    
# Instantiate the IronhackSpider class
my_spider = IronhackSpider(PAGES_TO_SCRAPE, content_parser=card_parser)

# Start scraping jobs
# my_spider.kickstart()

In [7]:
catalog = requests.get('http://www.starcitygames.com/catalog/magic_the_gathering/').content
catalog_web = BeautifulSoup(catalog, 'html')
links_to_categories = catalog_web.select('ul.cardset_lists a[href^= "http://www.starcitygames.com/catalog/category/"]')
links = [link['href'] for link in links_to_categories]
links



['http://www.starcitygames.com/catalog/category/10th Edition',
 'http://www.starcitygames.com/catalog/category/3rd EditionRevised',
 'http://www.starcitygames.com/catalog/category/4th Edition',
 'http://www.starcitygames.com/catalog/category/5th Edition',
 'http://www.starcitygames.com/catalog/category/6th Edition',
 'http://www.starcitygames.com/catalog/category/7th Edition',
 'http://www.starcitygames.com/catalog/category/8th Edition',
 'http://www.starcitygames.com/catalog/category/9th Edition',
 'http://www.starcitygames.com/catalog/category/Aether Revolt',
 'http://www.starcitygames.com/catalog/category/Alara Reborn',
 'http://www.starcitygames.com/catalog/category/Alliances',
 'http://www.starcitygames.com/catalog/category/Alpha',
 'http://www.starcitygames.com/catalog/category/Alternate 4th Edition',
 'http://www.starcitygames.com/catalog/category/Amonkhet',
 'http://www.starcitygames.com/catalog/category/Anthologies',
 'http://www.starcitygames.com/catalog/category/Antiquities'

In [93]:
def card_parser(content):
    table = pd.read_html(content)
    df_table = table[1]

    df_table = df_table.drop(0)
    name = df_table.iloc[0,]
    df_table = df_table.rename(name, axis = 1)
    df_table = df_table.drop(1)
    df_table = df_table.drop(['Mana', 'Stock'], axis = 1)

    #df_table.columns[7]
    df_table = df_table.drop(df_table.columns[7],  axis = 1)
    df_table = df_table.dropna(axis = 0, subset = ['Name', 'Category'])
    df_table = df_table.reset_index(drop = True)

    for doble_price in df_table.Price:
        fix_price = re.findall('..\...$', doble_price)
        df_table.Price = df_table.Price.replace({doble_price: fix_price})
    
    return df_table

card_parser = card_parser

'''def next_page(content)
    card_page = BeautifulSoup(content, 'html')
    next_link = card_page.select('section.box_sizing_fix#content div[style] a:contains(Next)')
    next_link = next_link[0]['href']
    return next_link
    '''

"def next_page(content)\n    card_page = BeautifulSoup(content, 'html')\n    next_link = card_page.select('section.box_sizing_fix#content div[style] a:contains(Next)')\n    next_link = next_link[0]['href']\n    return next_link\n    "

In [94]:
PAGES_TO_SCRAPE = links[0:2]

my_spider = IronhackSpider(PAGES_TO_SCRAPE, content_parser=card_parser)

my_spider.kickstart(2)